### Задание 1.
Определим функцию calc_PI для вычисления значения числа PI методом Монте-Карло

In [13]:
import numpy as np
import numba as nb


@nb.njit(parallel=True, fastmath=True)
def calc_PI(
    x0: np.float64, y0: np.float64, r0: np.float64, exp_num: np.int64
) -> np.float64:

    r0_pow_2 = np.power(r0, 2)
    x_max = x0 + r0
    x_min = x0 - r0
    y_max = y0 + r0
    y_min = y0 - r0

    xp = (x_max - x_min) * np.random.random(exp_num) + x_min
    yp = (y_max - y_min) * np.random.random(exp_num) + y_min

    return 4 * np.sum((xp - x0) ** 2 + (yp - y0) ** 2 < r0_pow_2) / exp_num


### Задание 2
Создадим 5 серий значений для количества экспериментов 10^4, 10^5, 10^6, 10^7, 10^8 и начальных параметров x0 = 1, y0 = 2, r0 = 5

In [16]:
x0 = 1
y0 = 2
r0 = 4

series = np.array(
    [
        np.array(
            [
                calc_PI(x0, y0, r0, exp_num)
                for exp_num in (10_000, 100_000, 1_000_000, 10_000_000, 100_000_000)
            ],
            dtype=np.float64,
        )
        for _ in range(5)
    ]
)

for i, c_series in enumerate(series):
    print(f"Series{i + 1}: {c_series}")


Series1: [3.1448     3.14576    3.141356   3.1412768  3.14182188]
Series2: [3.1076     3.14348    3.142136   3.1417896  3.14153956]
Series3: [3.148      3.13172    3.141076   3.1417004  3.14153308]
Series4: [3.1316     3.1352     3.143432   3.14121    3.14145148]
Series5: [3.1464    3.13472   3.141416  3.1421372 3.1416642]


### Задание 3.1
Рассчитаем погрешности для каждого значения в каждой серии

In [17]:
series_error = np.abs((series - np.pi) / np.pi)

for i, c_series_errors in enumerate(series_error):
    print(f"Series{i + 1}: {c_series_errors}")

Series1: [1.02093007e-03 1.32650756e-03 7.53291772e-05 1.00539320e-04
 7.29650325e-05]
Series2: [1.08201977e-02 6.00761021e-04 1.72952534e-04 6.26899894e-05
 1.69002145e-05]
Series3: [2.03952171e-03 3.14256324e-03 1.64455945e-04 3.42967476e-05
 1.89628626e-05]
Series4: [3.18076043e-03 2.03484484e-03 5.85482146e-04 1.21802421e-04
 4.49369493e-05]
Series5: [1.53022589e-03 2.18763358e-03 5.62305841e-05 1.73334506e-04
 2.27739297e-05]


### Задание 3.2
Рассчитаем погрешности по среднему результату

In [18]:
S_e = np.sum(series[:, :], axis=0) / len(series)
Eps_S_e = np.abs((S_e - np.pi) / np.pi)

for i, c_eps_s_e in enumerate(Eps_S_e):
    print(
        f"Среднее значение погрешности вычислений для 10^{4 + i} испытаний: {c_eps_s_e:.10e}"
    )


Среднее значение погрешности вычислений для 10^4 испытаний: 1.8820560912e-03
Среднее значение погрешности вычислений для 10^5 испытаний: 1.0875546153e-03
Среднее значение погрешности вычислений для 10^6 испытаний: 9.2483794764e-05
Среднее значение погрешности вычислений для 10^7 испытаний: 9.5959004019e-06
Среднее значение погрешности вычислений для 10^8 испытаний: 2.9877871646e-06


### Задание 4
1. Определим функцию calc_integral для определения значения интеграла методом Монте-Карло
2. Рассчитаем значение интеграла для функции $f(x) = x^3 + 1$
3. Рассчитаем погрешности для каждого эксперимента
4. Рассчитаем погрешности по средним результатам

Начальные условия: a = 0, b = 2, ExpNmb = 10^4, 10^5, 10^6, 10^7. Численное значение интеграла - 6

#### 4.1

In [2]:
from typing import Callable

import numba as nb
import numpy as np

@nb.njit(parallel=True, fastmath=True)
def calc_integral(
    func: Callable[[np.float64], np.float64],
    a: np.float64,
    b: np.float64,
    exp_num: np.int64,
) -> np.float64:
    m = 0

    x_min = a
    x_max = b
    y_min = 0
    y_max = func(b)

    for _ in nb.prange(exp_num):
        xp = (x_max - x_min) * np.random.random() + x_min
        yp = (y_max - y_min) * np.random.random() + y_min
        if yp <= func(xp):
            m += 1

    return (m / exp_num) * (b - a) * y_max


#### 4.2

In [5]:
a = 0
b = 2


@nb.njit(fastmath=True)
def func(x: np.float64) -> np.float64:
    return np.power(x, 3) + 1


series = np.array(
    [
        np.array(
            [
                calc_integral(func, a, b, exp_num)
                for exp_num in (10_000, 100_000, 1_000_000, 10_000_000)
            ],
            dtype=np.float64,
        )
        for _ in range(5)
    ]
)

for i, c_series in enumerate(series):
    print(f"Series{i + 1}: {c_series}")

Series1: [5.949      5.9562     5.981688   6.00017276]
Series2: [6.0786     5.98104    6.000246   5.99966638]
Series3: [5.9256     5.95314    5.988168   6.00049017]
Series4: [6.0534     5.9823     5.993298   5.99983546]
Series5: [5.994      5.95764    5.990634   6.00025457]


#### 4.3.1

In [ ]:
answer = 6

series_error = np.abs((series - answer) / answer)

for i, c_series_errors in enumerate(series_error):
    print(f"Series{i + 1}: {c_series_errors}")

Series1: [0.0071    0.00421   0.000502  0.0006133]
Series2: [1.400e-03 2.540e-03 1.418e-03 4.180e-05]
Series3: [0.0058    0.00314   0.001547  0.0005786]
Series4: [1.90e-03 2.74e-03 4.90e-04 6.37e-05]
Series5: [0.0037    0.00697   0.000763  0.0004889]


#### 4.3.2

In [ ]:
S_e = np.sum(series[:, :], axis=0) / len(series)
Eps_S_e = np.abs((S_e - answer) / answer)

for i, c_eps_s_e in enumerate(Eps_S_e):
    print(
        f"Среднее значение погрешности вычислений для 10^{4 + i} испытаний: {c_eps_s_e:.10e}"
    )


Среднее значение погрешности вычислений для 10^4 испытаний: 5.8000000000e-04
Среднее значение погрешности вычислений для 10^5 испытаний: 1.6480000000e-03
Среднее значение погрешности вычислений для 10^6 испытаний: 2.4200000000e-04
Среднее значение погрешности вычислений для 10^7 испытаний: 6.9740000000e-05
